# Document vectors
The first thing we're going to do, as usual, is begin by importing libraries and modules we're going to use today. We're introducing a new library, called ```datasets```, which is part of the ```huggingface``` universe. 

```datasets``` provides easy access to a wide range of example datasets which are widely-known in the NLP world, it's worth spending some time looking around to see what you can find. For example, here are a collection of [multilabel classification datasets](https://huggingface.co/datasets?task_ids=task_ids:multi-class-classification&sort=downloads).

We'll be working with the ```huggingface``` ecosystem more and more as we progress this semester.

In [1]:
# data processing
import pandas as pd
import numpy as np

# huggingface datasets
from datasets import load_dataset

# scikit learn tools
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

# plotting tools
import matplotlib.pyplot as plt

## Load data
We're going to be working with actual text data data, specifically a subset of the well-known [GLUE Benchmarks](https://gluebenchmark.com/). These benchmarks are regularly used to test how well certain models perform across a range of different language tasks. We'll work today specifically with the Stanford Sentiment Treebank 2 (SST2) - you can learn more [here](https://huggingface.co/datasets/glue) and [here](https://nlp.stanford.edu/sentiment/index.html).

The dataset we get back is a complex, hierarchical object with lots of different features. I recommend that you dig around a little and see what it contains. For today, we're going to work with only the training dataset right now, and we're going to split it into sentences and labels.

In [2]:
# load the sst2 dataset
dataset = load_dataset("glue", "sst2")
# select the train split
train_data = dataset["train"]
X = train_data["sentence"]
y = train_data["label"]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /home/coder/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Let's split the data into a training and a test set. We will later train a simple classifier to start looking at what one can do with vector representations of text, that's why we need a set of documents that are left aside. For now, let's simply focus on the training set to estimate our document-term model.

In [3]:
import random
train_idx = random.sample(range(len(X)), k=int(len(X)*.7)) # we are sampling 70% as training set
train_X, test_X, train_y, test_y = [], [], [], []
for i in train_idx:
    train_X.append(X[i])
    train_y.append(y[i])
for i in set(range(len(X))) - set(train_idx):
    test_X.append(X[i])
    test_y.append(y[i])

In [4]:
list(zip(train_X[:10], train_y[:10]))

[("a '' range ", 1),
 ('otherwise comic narrative ', 1),
 ('extraordinarily good ', 1),
 ('engages us in constant fits of laughter , until we find ourselves surprised at how much we care about the story , ',
  1),
 ('messy , uncouth , incomprehensible , vicious and absurd ', 0),
 ('robert john burke as the monster horns in and steals the show . ', 1),
 ('glee ', 1),
 ("filmmaking from one of french cinema 's master craftsmen ", 1),
 ('bible stores of the potential for sanctimoniousness , making them meaningful for both kids and church-wary adults ',
  1),
 ("it 's kind of insulting , both to men and women . ", 0)]

In [5]:
print('Number of training examples: ', len(train_X))
print('Number of test examples: ', len(test_X))

Number of training examples:  47144
Number of test examples:  20205



## Create document representations
We're going to work with a bag-of-words model (like the ones we talked about in class), which we can create quite simply using the ```CountVectorizer()``` class available via ```scikit-learn```. You can read more about the default parameters of the vectorizer [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

After we initialize the vectorizer, we first _fit_ this vectorizer to our data (the model learns parameters such as which words to include in the vocabulary, based on the statistics of the text and the parameters passed to  `CountVectorizer`) and then _transform_ the original data into the bag-of-words representation.

Let's start by fitting a model where default constraints are placed on vocabulary size.

In [6]:
simple_vectorizer = CountVectorizer()
X_vect = simple_vectorizer.fit_transform(train_X)

This is the number of words the vectorizer uses as features (i.e., words that are *not* excluded because too frequent, or too infrequent)

In [7]:
len(simple_vectorizer.vocabulary_)

13527

In [8]:
print(X_vect.shape)
print(X_vect.toarray())


(47144, 13527)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


As you can see, the resulting matrix has dimensions `[n_documents, n_words]`.
Note that there is a simple way to get a term-term matrix (in how many documents two words co-occur) by computing the dot product of the term-document matrix and its transpose.

In [9]:
np.dot(X_vect.T, X_vect).toarray() # the diagonal essentially indicates how often a term occurs overall.

array([[ 7,  7,  0, ...,  0,  0,  0],
       [ 7, 71,  0, ...,  0,  0,  0],
       [ 0,  0, 14, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  2,  0,  0],
       [ 0,  0,  0, ...,  0,  2,  0],
       [ 0,  0,  0, ...,  0,  0,  3]])

What happens to dimensionality if manipulate input parameters, e.g., `min_df`? Try to play with `CountVectorizer` parameters to get familiar with the function.

### Dimensionality reduction
Our current matrix is fairly sparse. Could we apply what we have learned during the lecture to convert it to a dense and more compact matrix? Let's apply the `SVD` algorithm we discussed in class.

In [10]:
svd = TruncatedSVD(n_components=500)
svd.fit(X_vect)
X_svd = svd.transform(X_vect)

How does our vector space look like?

In [11]:
X_svd

array([[ 1.28146041e-04,  4.87574862e-05,  4.58952640e-04, ...,
         8.38507497e-03, -2.09609040e-03, -2.19579164e-03],
       [ 8.90374404e-03,  3.89490112e-03, -5.52806602e-03, ...,
        -1.19276985e-02, -7.77091469e-02, -9.52588255e-03],
       [ 1.65737774e-02,  8.92747796e-03,  7.81190814e-03, ...,
        -1.32844170e-03,  7.65441054e-03,  3.73301160e-03],
       ...,
       [ 6.73435819e-03,  6.96980375e-03, -3.70214989e-03, ...,
        -2.97798296e-02,  1.77350045e-02,  5.83074169e-02],
       [ 1.56836293e+00,  2.70775459e-01, -7.71412492e-01, ...,
        -6.08606626e-02, -8.18754333e-03,  6.43568715e-02],
       [ 1.88089345e-01,  6.53942752e-02,  1.18022443e-01, ...,
        -4.79421002e-02,  3.86745226e-04, -1.60041111e-02]])

### Classifying sentiment

Congratulations! You have created your first document representation. 

We will dive deeper into classification in the coming weeks, but to demonstrate what we can do with these representations, let's go through an example.

As we saw earlier, our documents have labels indicating the sentiment of each of the document. Can we predict sentiment on the basis of bag of words representations of our documents?
Let's use a simple `scikit-learn` classifier to learn to predict sentiment from text. We will learn more about this later on, for now all you need to know is that the classifier estimates a relation between input and output such that it is able to predict the output (in this case, the sentiment of the sentence, which is `0` for negative sentences, `1` for positive) from the input.

We will use a `LogisticRegression` classifier (not necessarily best, but one the fastest), but you can experiment with multiple classifiers (e.g., https://scikit-learn.org/stable/modules/svm.html).

In [12]:
classifier = LogisticRegression(max_iter=2000).fit(X_vect, train_y)


Let's transform the test data, which we need for evaluation.

In [13]:
X_vect_test = simple_vectorizer.transform(test_X)

And finally, let's compute how often the model predictions match the true labels.

In [14]:
print('Model accuracy: ', np.mean(classifier.predict(X_vect_test) == test_y))

Model accuracy:  0.8941351150705271


That's pretty good: let's take a look at a couple of examples.

In [15]:
list(zip(test_X, classifier.predict(X_vect_test)))

[('that loves its characters and communicates something rather beautiful about human nature ',
  1),
 ('demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
  1),
 ("a depressed fifteen-year-old 's suicidal poetry ", 0),
 ("are more deeply thought through than in most ` right-thinking ' films ", 1),
 ("for those moviegoers who complain that ` they do n't make movies like they used to anymore ",
  0),
 ('the greatest musicians ', 1),
 ('cold movie ', 0),
 ('with his usual intelligence and subtlety ', 1),
 ('redundant concept ', 0),
 ('equals the original and in some ways even betters it ', 1),
 ('a smile on your face ', 1),
 ('enriched by an imaginatively mixed cast of antic spirits ', 1),
 ("which half of dragonfly is worse : the part where nothing 's happening , or the part where something 's happening ",
  0),
 ('sit through , ', 0),
 ("more than another `` best man '' clone by weaving 

### Some optional tasks
- Does performance change if we use a `TfidfVectorizer`?
- Can you write your own version of `CountVectorizer()`? In other words, a function that takes a corpus of documents and creates a bag-of-words representation for every document?
- What about `TfidfVectorizer()`? Look over the formulae in the slides from Tuesday.